<a href="https://colab.research.google.com/github/Kiki18-075/MusicGenreClassification/blob/main/MusicGenreClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Singkronisasi ke Google Drive**
Data yang digunakan pada program ini disimpan di google drive. untuk mengakses datanya perlu melakukan singkronisasi ke google drive terlebih dahulu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Import Library**
dalam program ini menggunakan beberapa library untuk mempermudah pekerjaan. library yang digunakan antara lain :
1. Pandas
2. Librosa
3. Scikit-Learn
4. numpy
5. os
6. glob

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os
import glob
import librosa
import librosa.feature
import librosa.display
from csv import writer

## **Ekstraksi fitur**
metode ekstraksi fitur yang digunakan dalam program ini adalah metode Mel Frequency Cesptral Coefficient (MFCC). dalam proses ekstraksi akan diambil nilai rata-rata dari proses ekstraksi sebanyak 26 nilai yang akan dijadikan fitur pada proses klasifikasi

In [ ]:
def mean_mfccs(x):
    return [np.mean(feature) for feature in librosa.feature.mfcc(x, n_mfcc=13)]

def parse_audio(x):
    return x.flatten('F')[:x.shape[0]] 

In [ ]:
def get_audios():    
    samples = []
    labels = []
    names = ['blues','classical','country','disco','hiphop','jazz','metal', 'pop','reggae', 'rock']
    
    for name in names:
        Music_folder = glob.glob('/content/drive/MyDrive/MusicGenreClassification/Data/'+name+'/*.wav')
        print('memproses %d data pada folder %s ...'%(len(Music_folder),name))
        for f in Music_folder:
            x, sr = librosa.load(f)
            x = parse_audio(x)
            samples.append(mean_mfccs(x))
            labels.append(name)
    return np.array(samples),np.array(labels)

In [ ]:
fitur, label= get_audios()

memproses 100 data pada folder blues ...
memproses 100 data pada folder classical ...
memproses 100 data pada folder country ...
memproses 100 data pada folder disco ...
memproses 100 data pada folder hiphop ...
memproses 100 data pada folder jazz ...
memproses 100 data pada folder metal ...
memproses 100 data pada folder pop ...
memproses 100 data pada folder reggae ...
memproses 100 data pada folder rock ...


## **Membuat Dataframe**
untuk membuat dataframe kita menggunakan library pandas. pada program ini terdapat 2 dataframe yaitu dataframe fitur dan dataframe label.

In [ ]:
dataframe_fitur = pd.DataFrame(fitur)
dataframe_label = pd.DataFrame(label)

setelah membuat dataframe kita bisa langsung melakukan pengecekan missing value

In [ ]:
dataframe_fitur.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64

dari nilai diatas dapat disimpulkan bahwa tidak ada missing value

## **Membagi Dataframe**
pada proses ini kita membagi data menjadi 2 bagian yang digunakan sebagai data training dan data testing dengan perbandingan 8:2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataframe_fitur, dataframe_label, test_size=0.2, random_state=42)

## **Proses Klasifikasi**
pada proses klasifikasi kita menggunakan metode K-Nearest Neighbor (KNN). 

proses pertama kita membuat fungsi untuk membuat model KNN yang dinamis untuk mencari nilai K yang optimal

In [ ]:
def KNN(k):

  knn = KNeighborsClassifier(n_neighbors = k)

  knn.fit(X_train,y_train)

  y_pred = knn.predict(X_test)
  print('K = ', i)
  print(classification_report(y_test,y_pred))


setelah membuat fungsi KNN kita melakukan uji performa dengan mencari nilai akurasi tertinggi dari nilai k ke-1 sampai nilai k ke-9

In [ ]:
for i in range (10):
  if ((i%2) != 0):
    KNN(i)

K =  1
              precision    recall  f1-score   support

       blues       0.67      0.60      0.63        20
   classical       0.90      0.69      0.78        13
     country       0.38      0.30      0.33        27
       disco       0.42      0.48      0.44        21
      hiphop       0.15      0.20      0.17        15
        jazz       0.43      0.59      0.50        22
       metal       0.76      0.64      0.70        25
         pop       0.30      0.54      0.39        13
      reggae       0.41      0.39      0.40        23
        rock       0.27      0.14      0.19        21

    accuracy                           0.45       200
   macro avg       0.47      0.46      0.45       200
weighted avg       0.47      0.45      0.45       200

K =  3
              precision    recall  f1-score   support

       blues       0.45      0.70      0.55        20
   classical       0.77      0.77      0.77        13
     country       0.30      0.33      0.32        27
       dis

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was

Nilai K Terbaik :
## **K-9 dengan akurasi 49%**